# Import Modules

In [5]:
# Import modules
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
from subprocess import Popen
import os
import pandas as pd
from tkinter import Entry

# Main window

In [6]:
# Create main window
window = Tk()
window.title("Data Cleaning Automation")
window.geometry("800x400")

''

# Upload File

In [7]:
def upload_file():
    global filename
    global sheet_names
    global file_name
    global Del_Colname_list
    global text_1
    
    curr_dir = os.getcwd() #Current directory
    filename = filedialog.askopenfilename(initialdir = curr_dir,title = "Select file from dialog",filetypes = ((".xlsx","*.xlsx"),(".csv","*.csv"),(".ipynb","*.ipynb")))
    load_file = pd.ExcelFile(filename) #Load Excel file
    sheet_names = load_file.sheet_names #Get Sheet names
    total_sheets = len(sheet_names) #Total No. of Sheet
   
    #GLobal
    global file
    global file_list
    global Row
    global Column
    global count_dtype
    global Blank_row
    global val
    global Blank_cells
    global BlankCol_list
    global Col_with_dtype
    global List_1
    global new_df
    global Colname_dtype
    global A_List
    global List_dtype
    global dtype_list
    global Blank_Column
    global dtype_df

    if total_sheets > 1:
        var = 0
        file = []
        for sheet in sheet_names:
            xl = load_file.parse(sheet_names[var],skiprows=0)
            row = xl.shape[0]
            if xl.empty and row <= 2:
                del xl
            else:
                pass
                file.append(xl)
            var += 1 
    else:
        file = pd.read_excel(load_file)
        
        
        
    #Split Filename from file path
    filename_list = filename.split('/')
    file_name = filename_list[-1] 
    
    
    #Column names into list
    file_list = file.columns

    #datatype of column into list
    dtype_list = file.dtypes
    dtype_df = pd.DataFrame({'Column':dtype_list.index, 'Dtype':dtype_list.values})
    
    
    
    # Summary
    Shape = file.shape
    Column = file.shape[1] #Total Column
    Row = file.shape[0]    #Total Row
    Blank_row = file.isnull().sum(axis=0) # Total blank cells in each column
    Blank_Column = file.isnull().sum(axis=1)
    count_dtype = file.dtypes.value_counts() # Column Name by data types
    

    
    #Check if Column contains blank cells
    Blank_cells = Blank_row[Blank_row != 0] 
    #Series_1 only contains columns which has blank cells
    Series_1 = Blank_cells 
    #Create dataframe for columns with blank cells
    new_df = pd.DataFrame({'Column':Series_1.index, 'Count':Series_1.values}) 

    #Merge Datatype from dtype_df after checking Same column names
    new_df2 = new_df.merge(dtype_df, left_index=True, on='Column', how='left')
      
    #Create List of dtypes of new_df2 
    List_dtype = []
    for i in new_df2['Dtype']:
        List_dtype.append(i)
                        
    #Create new list with Columns of blank cells
    List_1 = []
    for i in new_df['Column']:
        List_1.append(i)
        

    #Check if blank cell columns is in file_list
    A_List = [i for i in file_list if i in List_1] 
    
    
    #Concate Column name and its datatype    
    Colname_dtype = []
    for i in range(len(A_List)):
        c = str(A_List[i])+' ('+str(List_dtype[i])+')'
        Colname_dtype.append(c) 
       
      
    # Save dtypes of column into col_dtype
    Col_with_dtype = pd.DataFrame({'Column':dtype_list.index, 'Type':dtype_list.values})   
   
    
    # Textbox
    text_1 = Text(window,height=18,width=55)
    lbl_1 = Label(text="SUMMARY").place(x=342,y=10)
    
    text_1.insert(END, "**************OLD SUMMARY************* ")
    text_1.insert(END, "\nUploaded File: ")
    text_1.insert(END, file_name)
    
    text_1.insert(END, "\nSheet Names \t: ")
    text_1.insert(END, sheet_names)
    
    text_1.insert(END, "\nTotal Column : ")
    text_1.insert(END, Column)
    
    text_1.insert(END, "\nTotal Row  \t : ")
    text_1.insert(END, Row)
    
    text_1.insert(END, "\n\nTotal blank cells: \n")
    text_1.insert(END, Blank_cells)
   
    text_1.insert(END, "\n\nTotal Column by dtype: \n")
    text_1.insert(END, count_dtype)
    
    text_1.place(x=349,y=32)
       
    
    scroll = Scrollbar(window)
    scroll.pack(side=RIGHT, fill=Y)
    scroll.config(command=text_1.yview)
    text_1.config(yscrollcommand=scroll.set)

# View file

In [8]:
def view_file():
        msgbox_1 = messagebox.askquestion("View File","Do you want to view file")
        
        if msgbox_1 == 'yes':
            Popen(filename,shell=True)
        else:
            messagebox.showinfo('Return','You will now return')
            
button_1 = Button(window, text ="View file", width=14, command =view_file).place(x=225,y=26)

# Delete column

In [9]:
def delete_Col():
    msgbox_2 = messagebox.askquestion("Delete Column","Do you want to delete column?")
    
    global file
    global file2
    
    global Del_Colname_list
    
    if msgbox_2 == 'yes':
        sub = Toplevel(window)
        sub.title('Delete')
        
        scrollbar = Scrollbar(sub)
        scrollbar.pack(side=RIGHT, fill=Y)

        listbox = Listbox(sub,selectmode=MULTIPLE,yscrollcommand=scrollbar.set)
        
        
        #Drop Column where all the values are Nan
        file = file.dropna(axis=1,how='all') 
        Threshold = 85/100
        #Calculate percentage by product of column and range
        Perc = Threshold * Column 
        #Create an empty list
        new_list = [] 
        for index,column in enumerate(Blank_Column):
             if column > Perc:
                new_list.append(index)

        file = file.drop(new_list) #Drop the empty row
        file_list2=file.columns
        
        #Insert Colname in Listbox
        for item in file_list2:
            listbox.insert(END, item)
        listbox.pack(side=LEFT, fill=BOTH)

        scrollbar.config(command=listbox.yview)
  
        res_list = [] 
        def delete():
            global file
            global col_after_del
            selection = listbox.curselection() 
            for i in selection:
                entr = listbox.get(i) #entr prints column name & i prints index no.
                res_list.append(entr) # reslist appends entr into list
            for val in res_list:
                    print(val)
                    
            file = file.drop(columns=res_list, axis=1)
            col_after_del = file.columns
            
            #Summary After deletion
            New_Column = file.shape[1]
            New_Row = file.shape[0]
           
            #Get undeleted columns
            Del_Colname_list = [i for i in A_List if i in col_after_del]
            
            
            #Check if column has been deleted or notby its length
            if len(col_after_del)<len(file_list):
                messagebox.showinfo("Deleted","Columns"+" "+str(res_list)+" has been deleted successfully!!")
            else:
                pass
       
    
        #Button to delete columns     
        button_3 = Button(sub, text="Delete", command=delete)
        button_3.pack(side=RIGHT)
              
    else: 
        col_after_del = file_list
        messagebox.showinfo('Return','You will now return to the application screen')

# Rename column

In [10]:
def rename_Col():
    global entry2
    global val
    global string
    global len_max  
    global Del_Colname_list
    global col_after_del
    global col_after_rename
    
    msgbox_3 = messagebox.askquestion("Rename column","Do you want to rename column names?")
    
    if msgbox_3 == 'yes':
        sub2 = Toplevel(window)
        sub2.title('Rename Window')
        sub2.geometry("400x200")
        
        scrollbar = Scrollbar(sub2)
        scrollbar.pack(side=RIGHT, fill=Y)
        

        listbox = Listbox(sub2, selectmode=MULTIPLE, yscrollcommand=scrollbar.set)
        
        for item in col_after_del:
            listbox.insert(END, item)
        listbox.pack(side=LEFT, fill=BOTH)

        scrollbar.config(command=listbox.yview) 
        
       
        label2 = Label(sub2, text="New Column Name:").place(x=232,y=3)
        
        entry2 = Entry(sub2) # Entry box to get value
        entry2.place(x=236,y=19)
  
           
        def rename(): 
            global val
            global string
            global entry2
            global res_list
            global file
            global col_after_rename
            global Colname_dtype2
            
            string = entry2.get().split(",")
            print(string)
            tup_2 = tuple(string) # Create tuple of value inserted by user
        
            
            res_list = [] # Empty list
            selection = listbox.curselection() 
            for i in selection:
                entr = listbox.get(i)
                res_list.append(entr)
            for val in res_list:
                print(val)
                tup_1 = tuple(res_list) # Create tuple of column names
        
            
                col_rename_dict = {i:j for i,j in zip(tup_1,tup_2)} #Create a dictionary
                file = file.rename(columns=col_rename_dict) #Rename columns
                col_after_rename = file.columns #Column Names after changes
                Col_rename_dtypes = file.dtypes #Dtype of Columns 
                Col_rename_df = pd.DataFrame({'Column':Col_rename_dtypes.index, 'Dtype':Col_rename_dtypes.values})#Create dataframe
                
                List_dtype_2 = []
                for i in Col_rename_df['Dtype']:
                    List_dtype_2.append(i)
                      
                    
                Colname_dtype2 = []
                for i in range(len(col_after_rename)):
                    c = str(col_after_rename[i])+' ('+str(List_dtype_2[i])+')'
                    Colname_dtype2.append(c)
                    print(Colname_dtype2)
                
                if len(col_after_rename)==len(Colname_dtype2):
                    messagebox.showinfo("Renamed","Columns"+" "+str(res_list)+" has been renamed successfully!!")
                else:
                    print("Nope")
                    
        # Button to rename column names
        btn1 = Button(sub2, text="Rename", command=rename)
        btn1.pack(side=RIGHT)
        
    else:
        messagebox.showinfo('Return','You will now return to the application screen')    

# Fill Column

In [11]:
def fill_Col():
    global string
    global e
    global file_columns
    global j
    j = []
    global Del_Colname_list
    global col_after_rename
    global Colname_dtype2
    
    MsgBox_3 = messagebox.askquestion("Fill column","Do you want to fill column?")
    if MsgBox_3 == 'yes':
        sub2 = Toplevel(window) #Subwindow of fill column
        sub2.title('Fill Column')
        sub2.geometry("600x550")
        
        # Entry box
        e = Entry(sub2)
        e.place(x=440, y=42)
        
        col_after_del_Series = pd.Series(col_after_rename)#Convert List into series
        A1 = col_after_del_Series
        A1_df = pd.DataFrame({'Column':A1.values}) #New dataframe with Updated Column Name list
        new_df3 = A1_df.merge(dtype_df, left_index=True, on='Column', how='left')
        
    
        def chkbox_checked(text):
            return lambda : j.append(text)
        
          
        # Checkboxes for Column name
        for val in Colname_dtype2:
                c = Checkbutton(sub2, text=val, command=chkbox_checked(val))
                c.grid(sticky=W)
          

        # Fill Mean Value in Blank cells
        def fill_mean():
            global j
            
            Col_split = [i.split(' (')[0] for i in j] # Split selected columns
            
            for i in Col_split:
                file[i].fillna((file[i].mean()),inplace =True)
                print(file[i])
                
            messagebox.showinfo("Filled","Columns"+" "+str(res_list)+" has been Filled by Mean")    
        
        # Fill Value in Blank cells    
        def fill_value():
            global e
            global string
            global j
            
            string = e.get() # Get value from user to fill blank cells
            print(string)
            
            Col_split = [i.split(' (')[0] for i in j] # Split selected columns
             
            for i in Col_split:
                file[i].fillna(string,inplace =True) # Fillna with value given by user
                print(file[i])  
                
            messagebox.showinfo("Filled","Columns"+" "+str(res_list)+" has been Filled by Value")    
            
        # Fill Maximum value        
        def fill_max():
            global j
            
            Col_split = [i.split(' (')[0] for i in j] # Split selected columns
            
            for i in Col_split:
                maxm = file[i].mode()
                print(maxm)
                
                maxm_df = pd.DataFrame({'Sr.no':maxm.index, 'Value':maxm.values})
                
                for j in maxm_df['Value']:
                    m = j
                    file[i].fillna(m,inplace =True) # Fillna with value given by user
                    print(file[i])            
          
            messagebox.showinfo("Filled","Columns"+" "+str(res_list)+" has been Filled by Max")
        
        def ok():
            print(file)
            Shape = file.shape
            Column = file.shape[1] #Total Column
            Row = file.shape[0]    #Total Row
            Blank_row = file.isnull().sum(axis=0) # Total blank cells in each column
            Blank_Column = file.isnull().sum(axis=1)
            count_dtype = file.dtypes.value_counts() # Column Name by data types
                

            #Textbox
            text_1.insert(END, "\n\n*********NEW SUMMARY******************")
    
            text_1.insert(END, "\nTotal Column : ")
            text_1.insert(END, Column)

            text_1.insert(END, "\nTotal Row  \t : ")
            text_1.insert(END, Row)

            text_1.insert(END, "\n\nTotal blank cells: \n")
            text_1.insert(END, Blank_row)

            text_1.insert(END, "\n\nTotal Column by dtype: \n")
            text_1.insert(END, count_dtype)

            messagebox.showinfo("Ok","Columns has been Filled Successfully!!")
  
        
        #Button to fill Mean Value
        button_12 = Button(sub2, text='Fill Mean',width=10, command = fill_mean).place(x=352, y=12)
                
        # Buttton to fill Value from user
        button_13 = Button(sub2, text='Fill Value',width=10, command =fill_value).place(x=352, y=42)
        
        # Button to fill Max. value
        button_14 = Button(sub2, text='Fill Max',width=10, command =fill_max).place(x=352, y=72)
        
        # Button of OK
        button_15 = Button(sub2, text='OK',width=10, command =ok).place(x=152, y=212)
        
    
    else: 
        messagebox.showinfo('Return','You will now return to the application screen')

# Save File

In [12]:
#Save File
def save_file():
    MsgBox_4 = messagebox.askquestion("Save File","Do you want to Save File?")
    
    if MsgBox_4 == 'yes':
        file.to_excel('DCA'+"__"+file_name, index=False)
        curr_dire = os.getcwd()
        messagebox.showinfo("File Saved","The current directory where File has been saved is :- "+curr_dire)
        
    else:
        messagebox.showinfo('Return','Thank you!!')

# Buttons

In [13]:
# Button to upload file 
button_6 = Button(window, text ="Upload file", width=20, command =upload_file).place(relx=0.02, rely=0.1, anchor=W)

# Button to delete column
button_7 = Button(window, text ="Delete Column", width=20, command =delete_Col).place(relx=0.02, rely=0.3, anchor=W)

# Button to rename column
button_8 = Button(window, text ="Rename Column", width=20, command =rename_Col).place(relx=0.02, rely=0.5, anchor=W)

# Button to fill column
button_9 = Button(window, text ="Fill Column", width=20, command =fill_Col).place(relx=0.02, rely=0.7, anchor=W)

# Button to save file
button_10 = Button(window, text='Save file',width=8, command =save_file).place(x=780, y=399, anchor=SE)

In [14]:
window.mainloop()